## Transformación y limpieza del dataset ##

En este notebook realizamos el proceso de transformación del dataset bruto, aplicando
las funciones definidas en `cleaning.py`.

El objetivo es obtener una versión limpia, coherente y analíticamente fiable del dataset,
siguiendo las conclusiones extraídas en la fase de exploración (EDA).

Este proceso se realiza sobre una copia del dataset original, que nunca se modifica.


In [33]:
import pandas as pd
import sys

# Añadimos la carpeta src/ al path para importar cleaning.py
sys.path.append("../src/")

import importlib, cleaning
importlib.reload(cleaning)

from cleaning import clean_data





1. Carga del dataset original

En este paso cargamos el dataset sin modificarlo.  
El dataset original se conserva intacto en la carpeta `data/`.

Toda la limpieza se aplicará posteriormente sobre una copia interna del DataFrame,
siguiendo el principio de trabajo seguro recomendado en los apuntes del curso.


In [34]:
df_raw = pd.read_csv("../data/movie_metadata.csv")
df_raw.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


2. Aplicación del pipeline de limpieza

Utilizamos la función `clean_data(df)` del módulo `cleaning.py`, que ejecuta una serie
de pasos automatizados y coherentes con el EDA:

# Justificación del proceso
El pipeline de limpieza incluye:

- **Estandarización global de columnas**
  - Todas las columnas textuales y categóricas se normalizan (strip + title case),
    excepto `content_rating`, que requiere un tratamiento especial.
- **Corrección general de tipos**
  - Años como enteros (Int32), categorías donde corresponde, y variables numéricas
    convertidas de forma segura.
- **Tratamiento razonado de valores nulos**
  - `language` → imputación con la MODA (English), justificado porque:
    - El idioma no puede ser nulo.
    - La moda representa la mayoría absoluta.
    - No imputarlo supondría eliminar información útil.
  - `country` → imputación con la MODA (USA), coherente con el idioma predominante.
- **Normalización de `content_rating`**
  - Unificación de códigos (PG-13, R, G, etc.) sin alterar su significado.
- **Extracción de `main_genre`**
  - Para facilitar análisis comparativos posteriores.
- **Creación de `movie_type`**
  - Clasificamos cada película en:
    - Cortometraje (< 60 min)
    - Película (60–120 min)
    - Pelicula_larga (> 120 min)
- **Corrección de outliers temporales**
  - Eliminamos años irreales (fuera del rango 1920–2016).

Este proceso garantiza que el dataset quede consistente, estandarizado y analíticamente útil.


In [35]:
df_clean = clean_data(df_raw)
df_clean.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,main_genre,movie_type
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,Usa,PG-13,237000000.0,2009,936.0,7.9,1.78,33000,Action,Pelicula_Larga
26,Color,James Cameron,315.0,194.0,0.0,794.0,Kate Winslet,29000.0,658672302.0,Drama|Romance,...,Usa,PG-13,200000000.0,1997,14000.0,7.7,2.35,26000,Drama,Pelicula_Larga
29,Color,Colin Trevorrow,644.0,124.0,365.0,1000.0,Judy Greer,3000.0,652177271.0,Action|Adventure|Sci-Fi|Thriller,...,Usa,PG-13,150000000.0,2015,2000.0,7.0,2.00,150000,Action,Pelicula_Larga
17,Color,Joss Whedon,703.0,173.0,0.0,19000.0,Robert Downey Jr.,26000.0,623279547.0,Action|Adventure|Sci-Fi,...,Usa,PG-13,220000000.0,2012,21000.0,8.1,1.85,123000,Action,Pelicula_Larga
66,Color,Christopher Nolan,645.0,152.0,22000.0,11000.0,Heath Ledger,23000.0,533316061.0,Action|Crime|Drama|Thriller,...,Usa,PG-13,185000000.0,2008,13000.0,9.0,2.35,37000,Action,Pelicula_Larga


3. Verificación del dataset limpio

Tras aplicar el pipeline, revisamos el dataset resultante para:

- Comprobar la correcta aplicación de las transformaciones.
- Validar la imputación de nulos.
- Confirmar que las nuevas columnas existen y tienen el formato esperado.
- Verificar el tipo de datos final y la ausencia de duplicados.

Esta inspección garantiza que la fase de transformación se ha completado correctamente.


In [36]:
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 4997 entries, 0 to 5040
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   color                      4997 non-null   category
 1   director_name              4997 non-null   object  
 2   num_critic_for_reviews     4948 non-null   float64 
 3   duration                   4982 non-null   float64 
 4   director_facebook_likes    4894 non-null   float64 
 5   actor_3_facebook_likes     4974 non-null   float64 
 6   actor_2_name               4997 non-null   object  
 7   actor_1_facebook_likes     4990 non-null   float64 
 8   gross                      4124 non-null   float64 
 9   genres                     4997 non-null   object  
 10  actor_1_name               4997 non-null   object  
 11  movie_title                4997 non-null   object  
 12  num_voted_users            4997 non-null   int64   
 13  cast_total_facebook_likes  4997 non-nu

4. Guardar el dataset limpio

Guardamos el dataset procesado en la carpeta:

📁 `data/processed/movie_clean.csv`

De este modo:

- Mantendremos siempre disponible el dataset original.
- Podremos reutilizar la versión limpia sin repetir la limpieza.





In [37]:
import os

processed_path = "../data/processed/"
os.makedirs(processed_path, exist_ok=True)

output_file = processed_path + "movie_clean.csv"
df_clean.to_csv(output_file, index=False)

output_file


'../data/processed/movie_clean.csv'

5. Conclusión de la fase de transformación

He creado una versión totalmente limpia del dataset, lista para análisis posteriores.

El proceso aplicado:

- Estandariza y homogeniza toda la información.
- Corrige tipos y categorías.
- Elimina duplicados.
- Trata valores nulos de manera razonada y justificada.
- Añade información derivada útil (`main_genre`, `movie_type`).
- Alinea el dataset con las buenas prácticas enseñadas en clase.

Con esta base sólida, podemos pasar al siguiente notebook para realizar
las visualizaciones obligatorias y extraer conclusiones exploratorias de calidad.
